In [3]:
import pandas as pd
import csv
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import joblib
from src.util import *

In [4]:
submission_table = pd.read_csv('top_submission_info.csv')
submission_table.head()

,Unnamed: 0,nsfw,score,subreddit_name,text,title
0,0,False,221852,AskReddit,NaN,"People who haven't pooped in 2019 yet, why are..."
1,1,False,175335,AskReddit,As many of you know today is day that many of ...,Stan Lee has passed away at 95 years old
2,2,False,160309,AskReddit,NaN,"Reddit, how would you feel about a law that ba..."
3,3,False,144660,AskReddit,,"Without saying what the category is, what are ..."
4,4,False,141175,AskReddit,NaN,"Anthony Bourdain once said ""There's a guy in m..."


In [5]:
submission_table = submission_table.drop(columns='Unnamed: 0')

In [6]:
submission_table.head()

,nsfw,score,subreddit_name,text,title
0,False,221852,AskReddit,NaN,"People who haven't pooped in 2019 yet, why are..."
1,False,175335,AskReddit,As many of you know today is day that many of ...,Stan Lee has passed away at 95 years old
2,False,160309,AskReddit,NaN,"Reddit, how would you feel about a law that ba..."
3,False,144660,AskReddit,,"Without saying what the category is, what are ..."
4,False,141175,AskReddit,NaN,"Anthony Bourdain once said ""There's a guy in m..."


In [5]:
# def tokenize(doc):
#     return [token for token in simple_preprocess(doc) if token not in STOPWORDS]

In [7]:
tfidf = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2))
sparse = tfidf.fit_transform(submission_table['title'])

In [8]:
joblib.dump(tfidf, 'tfvect.joblib')

['tfvect.joblib']

In [9]:
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())
dtm.head()

,aa,aa batteries,abandon,abandon twins,abandoned,abandoned walmart,abandonment,abandonment terry,abducted,abducted father,...,ｃｈａｎｇｅｄ ｍｙ,ｆｏｎｔ,ｆｏｎｔ ｔｏ,ｍｙ,ｍｙ ｆｏｎｔ,ｔｈｉｓ,ｔｏ,ｔｏ ｔｈｉｓ,ｖｅ,ｖｅ ａｃｃｉｄｅｎｔａｌｌｙ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
nn = NearestNeighbors(n_neighbors=5, algorithm='brute')
nn.fit(dtm)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [11]:
joblib.dump(nn, 'nn.joblib')

['nn.joblib']

In [11]:
query = ["""This cat's peculiar jumping style."""]

In [12]:
looking = tfidf.transform(query)
nn.kneighbors(looking.todense())

(array([[0., 1., 1., 1., 1.]]),
 array([[1675,  567, 1344, 1180,  562]], dtype=int64))

In [13]:
for i in nn.kneighbors(looking.todense())[1]:
    print(submission_table['subreddit_name'][i])

1675    oddlysatisfying
567          Unexpected
1344       HistoryMemes
1180       comedyheaven
562          Unexpected
Name: subreddit_name, dtype: object
